In [1]:
import copy
import os

import tensorflow as tf
import keras_tuner as kt


class DeepTuner(kt.Tuner):
    def run_trial(self, trial, X, y, validation_data, **fit_kwargs):
        model = self.hypermodel.build(trial.hyperparameters)

        model.fit(
            X,
            y,
            batch_size=trial.hyperparameters.Choice("batch_size", [16, 32]),
            **fit_kwargs
        )

        X_val, y_val = validation_data  # get the validation data
        eval_scores = model.evaluate(X_val, y_val)
        self.save_model(trial.trial_id, model)  # save the model to disk
        return {
            name: value for name, value in zip(model.metrics_names, eval_scores)
        }  # inform the oracle of the eval result, the result is a dictionary with the metric names as the keys.

    def save_model(self, trial_id, model, step=0):
        fname = os.path.join(self.get_trial_dir(trial_id), "model")
        model.save(fname)

    def load_model(self, trial):
        fname = os.path.join(self.get_trial_dir(trial.trial_id), "model")
        model = tf.keras.models.load_model(fname)
        return model



In [2]:
from sklearn.datasets import load_digits

# Load the hand-written digits dataset
digits = load_digits()

# Get the images and corresponding labels
images, labels = digits.images, digits.target
images.shape, labels.shape

# reshape images to vectors
n_samples = len(images)
X = images.reshape((n_samples, -1))

# Split data into train and test subsets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, labels, test_size=0.2, shuffle=False
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, shuffle=False
)
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_train[:10])


(1149, 64)
(1149,)
(360, 64)
[0 1 2 3 4 5 6 7 8 9]


In [3]:
import kerastuner as kt


def build_model(hp):
    model = tf.keras.Sequential()
    model.add(tf.keras.Input(shape=(64,)))
    for i in range(hp.Int("num_layers", min_value=1, max_value=4)):
        model.add(
            tf.keras.layers.Dense(
                hp.Int("units_{i}".format(i=i), min_value=32, max_value=128, step=32),
                activation="relu",
            )
        )
    model.add(tf.keras.layers.Dense(10, activation="softmax"))
    model.compile(loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model


random_tuner = DeepTuner(
    oracle=kt.oracles.RandomSearch(
        objective=kt.Objective("accuracy", "max"), max_trials=10, seed=42
    ),
    hypermodel=build_model,
    overwrite=True,
    project_name="random_tuner",
)

random_tuner.search(X_train, y_train, validation_data=(X_val, y_val), epochs=10)
random_tuner.search_space_summary()


Trial 10 Complete [00h 00m 03s]
accuracy: 0.9375

Best accuracy So Far: 0.9583333134651184
Total elapsed time: 00h 00m 46s
Search space summary
Default search space size: 6
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 4, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 32, 'sampling': 'linear'}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 32, 'sampling': 'linear'}
units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 32, 'sampling': 'linear'}
batch_size (Choice)
{'default': 16, 'conditions': [], 'values': [16, 32], 'ordered': True}
units_3 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 32, 'sampling': 'linear'}


In [4]:
best_model = random_tuner.get_best_models(1)[0]
y_pred_test = best_model.evaluate(X_test, y_test)
print(best_model.metrics_names)
print(y_pred_test)


12/12 [==============================] - 0s 3ms/step - loss: 0.5006 - accuracy: 0.9000
['loss', 'accuracy']
[0.5005685687065125, 0.8999999761581421]
